# Reversed Metaphone using Keras seq2seq

This is a quick demo just for fun, demonstrating a Keras seq2seq model 'translating' from Double Metaphone back to German surnames.

This notebook based on the Keras example found at https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py. The only changes are the data (now a set of 8000 German surnames) and the use of Double Metaphone to transform those surnames.

In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

from abydos.phonetic import DoubleMetaphone
dm = DoubleMetaphone()

Using TensorFlow backend.


In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

The data is the `nachnamen.csv` file from the tests/corpora directory. Only the first field, which contains the surnames, is used.

In [3]:
data_path = '../tests/corpora/nachnamen.csv'

Below, as each line is read from the file, the first field is retained and the first Double Metaphone encoding is calculated. For training, the Double Metaphone is the input and the original name is the target value.

In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    target_text = line.split(',')[0]
    input_text = dm.encode(target_text)[0]
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [6]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 9999
Number of unique input tokens: 14
Number of unique output tokens: 60
Max sequence length for inputs: 9
Max sequence length for outputs: 17


In [7]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [8]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [9]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [10]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [11]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [12]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

Train and save the model below:

In [13]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 7999 samples, validate on 2000 samples
Epoch 1/100
7999/7999 [==============================] - 5s 568us/step - loss: 1.1993 - val_loss: 1.1734
Epoch 2/100
7999/7999 [==============================] - 3s 387us/step - loss: 0.9898 - val_loss: 0.9711
Epoch 3/100
7999/7999 [==============================] - 3s 403us/step - loss: 0.8165 - val_loss: 0.8593
Epoch 4/100
7999/7999 [==============================] - 3s 399us/step - loss: 0.7105 - val_loss: 0.7540
Epoch 5/100
7999/7999 [==============================] - 3s 394us/step - loss: 0.6313 - val_loss: 0.6997
Epoch 6/100
7999/7999 [==============================] - 3s 393us/step - loss: 0.5714 - val_loss: 0.6377
Epoch 7/100
7999/7999 [==============================] - 3s 383us/step - loss: 0.5225 - val_loss: 0.6051
Epoch 8/100
7999/7999 [==============================] - 3s 384us/step - loss: 0.4849 - val_loss: 0.5712
Epoch 9/100
7999/7999 [==============================] - 3s 380us/step - loss: 0.4536 - val_loss: 0.5625
Epoch 1

Epoch 78/100
7999/7999 [==============================] - 3s 412us/step - loss: 0.1301 - val_loss: 0.6309
Epoch 79/100
7999/7999 [==============================] - 3s 411us/step - loss: 0.1288 - val_loss: 0.6341
Epoch 80/100
7999/7999 [==============================] - 3s 421us/step - loss: 0.1289 - val_loss: 0.6297
Epoch 81/100
7999/7999 [==============================] - 3s 418us/step - loss: 0.1286 - val_loss: 0.6339
Epoch 82/100
7999/7999 [==============================] - 3s 413us/step - loss: 0.1279 - val_loss: 0.6371
Epoch 83/100
7999/7999 [==============================] - 3s 435us/step - loss: 0.1276 - val_loss: 0.6338
Epoch 84/100
7999/7999 [==============================] - 4s 451us/step - loss: 0.1270 - val_loss: 0.6420
Epoch 85/100
7999/7999 [==============================] - 4s 446us/step - loss: 0.1265 - val_loss: 0.6409
Epoch 86/100
7999/7999 [==============================] - 3s 405us/step - loss: 0.1263 - val_loss: 0.6415
Epoch 87/100
7999/7999 [======================

/home/chrislit/fast/anaconda3/envs/tf/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [14]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [15]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [16]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

Finally, some decoded examples are shown below. The decoded sequences are generally German-like surnames (some quite common) and are names that would be encoded to the input sequence by Double Metaphone.

In [17]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('Input sequence:', input_texts[seq_index])
    print('Decoded sequence:', decoded_sentence)

Input sequence: MLR
Decoded sequence: Meller

Input sequence: XMT
Decoded sequence: Schmitt

Input sequence: XNTR
Decoded sequence: Schneider

Input sequence: FXR
Decoded sequence: Fischer

Input sequence: APR
Decoded sequence: Wieber

Input sequence: MR
Decoded sequence: Mayer

Input sequence: AKNR
Decoded sequence: Wagner

Input sequence: PKR
Decoded sequence: Böker

Input sequence: XLS
Decoded sequence: Scholze

Input sequence: HFMN
Decoded sequence: Hofmann

Input sequence: XFR
Decoded sequence: Schiefer

Input sequence: KK
Decoded sequence: Kuch

Input sequence: PR
Decoded sequence: Bauer

Input sequence: RKTR
Decoded sequence: Richter

Input sequence: KLN
Decoded sequence: Klein

Input sequence: XRTR
Decoded sequence: Schröder

Input sequence: ALF
Decoded sequence: Wulff

Input sequence: NMN
Decoded sequence: Neumann

Input sequence: XRS
Decoded sequence: Schwarz

Input sequence: SMRMN
Decoded sequence: Zimmermann

Input sequence: PRN
Decoded sequence: Braun

Input sequence: HFMN